In [1]:
import requests
import json
import pandas as pd

# Items

In [48]:
response=requests.get("https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/item-data.json")
item_list = json.loads(response.content.decode("utf-8"))
item_df = pd.DataFrame.from_dict(item_list).astype({"price": int})

In [49]:
# ingredients would be better to unified with lower case for comparing purpose
item_df["ingredients"] = item_df["ingredients"].str.lower()

In [50]:
item_df.dtypes

category        object
gender          object
id               int64
imageId         object
ingredients     object
monthlySales     int64
name            object
price            int64
dtype: object

In [74]:
# to check proper column length
for column in item_df.columns:
    if item_df[column].dtype != object:
        continue
    print (column, item_df[column].map(len).max())

category 10
gender 6
imageId 36
ingredients 96
name 70


In [83]:
item_list[4]["imageId"]

'1647f43c-2919-4cbf-9de4-b56a4817779d'

In [52]:
# to prepare for one to many table
item_ingr_df = item_df[["id", "ingredients"]]

# Ingredients

In [65]:
response=requests.get("https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/ingredient-data.json")
ingr_list = json.loads(response.content.decode("utf-8"))
ingr_df = pd.DataFrame.from_dict(ingr_list)


In [66]:
# unify with lower case as above
ingr_df["name"] = ingr_df["name"].str.lower()

In [67]:
# index column would be pkey for ingredient table
ingr_df = ingr_df.sort_values(by="name").reset_index(drop=True)

In [68]:
def convert_score(mark):
    if mark == "O":
        return 1
    elif mark == "X":
        return -1
    elif mark == "":
        return 0
    else:
        raise ValueError("Unexpected mark: {}".format(mark))
        
for column in ingr_df.columns:
    if column == "name":
        continue
    ingr_df[column] = ingr_df[column].map(convert_score)

In [72]:
# check max len for proper column len
print (ingr_df["name"].map(len).max())

18


# Item-Ingredients (one to many)

In [58]:
# ingredient - primary key dict
ingr_pkey_dict = dict(zip(ingr_df["name"], ingr_df.index))

In [59]:
itemkey_ingrkey_list = list()
unknown_ingr_list = list()

for i in item_ingr_df.index:
    row = item_ingr_df.loc[i]
    ingredients = row["ingredients"].split(",")
    
    # remove potential duplicates
    ingredients = list(set(ingredients))
    
    for ingr in ingredients:
        pkey = ingr_pkey_dict.get(ingr)
        if pkey is None:
            unknown_ingr_list.append(ingr)
        else:
            itemkey_ingrkey_list.append((row["id"], pkey))

if len(unknown_ingr_list):
    print ("Unknown ingredient list: {}".format(len(unknown_ingr_list)))

In [61]:
len(itemkey_ingrkey_list)

5550

In [73]:
itemkey_ingrkey_list

[(1, 361),
 (1, 277),
 (1, 614),
 (1, 823),
 (1, 736),
 (2, 460),
 (2, 254),
 (2, 247),
 (2, 34),
 (2, 984),
 (2, 353),
 (3, 976),
 (3, 836),
 (3, 685),
 (3, 359),
 (4, 160),
 (4, 668),
 (4, 164),
 (4, 956),
 (5, 125),
 (5, 178),
 (5, 586),
 (5, 453),
 (5, 226),
 (5, 673),
 (6, 25),
 (6, 445),
 (6, 671),
 (7, 163),
 (7, 678),
 (7, 383),
 (7, 62),
 (7, 864),
 (8, 613),
 (8, 218),
 (8, 707),
 (8, 487),
 (8, 314),
 (8, 22),
 (9, 212),
 (9, 614),
 (9, 771),
 (9, 610),
 (9, 595),
 (10, 122),
 (10, 233),
 (10, 624),
 (10, 587),
 (10, 943),
 (10, 751),
 (10, 91),
 (11, 868),
 (11, 364),
 (11, 821),
 (11, 120),
 (11, 740),
 (11, 448),
 (11, 60),
 (11, 230),
 (12, 782),
 (12, 977),
 (12, 180),
 (13, 899),
 (13, 750),
 (13, 279),
 (13, 425),
 (13, 562),
 (14, 875),
 (14, 206),
 (14, 964),
 (14, 588),
 (15, 731),
 (15, 623),
 (15, 751),
 (15, 287),
 (15, 736),
 (16, 664),
 (16, 718),
 (16, 123),
 (16, 175),
 (16, 806),
 (17, 770),
 (17, 373),
 (17, 127),
 (17, 386),
 (17, 936),
 (17, 300),
 (17, 